In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import networkx as nx

In [104]:
cb1 = pd.read_csv('cb1.csv')
cb2 = pd.read_csv('cb2.csv')
cb3 = pd.read_csv('cb3.csv')
cb = cb1.append(cb2)
cb = cb.append(cb3)
cb.index = range(len(cb))
cb['date_my'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('m')
cb['date_y'] = (pd.to_datetime(cb['date'],format='%m/%d/%Y', errors='coerce')).dt.to_period('y')

nodes_org = cb[['target_name', 'target_ID', 'target_country', 'target_continent','industry']].drop_duplicates()
nodes_org['bipartite'] = 1
nodes_inv = cb[['investors_name', 'investor_id', 'investor_country', 'investor_continent', 'investor_type']].drop_duplicates()
nodes_org = nodes_org.rename(columns={"target_name": "node", "target_country": "country", 
                                      "target_continent": "continent", "industry":"industry",
                                     "target_ID":"index"})
nodes_inv = nodes_inv.rename(columns={"investors_name": "node","investor_country": "country", 
                                      "investor_continent": "continent", "investor_type":"industry","investor_id":'index' })
nodes_inv['bipartite'] = 0
nodes = nodes_org.append(nodes_inv, ignore_index= True)
nodes['id'] = nodes.index

numbers = cb.groupby(['target_name','stage'], as_index = False).count()[['target_name','stage','investors_name']].rename(columns={"investors_name": "investor_numbers"})
cb_new = pd.merge(cb, numbers,  how='left', left_on=['target_name','stage'], right_on = ['target_name','stage'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 1],  how='left', left_on=['target_name','target_ID'], right_on = ['node','index'])
cb_new = pd.merge(cb_new, nodes[nodes['bipartite']== 0],  how='left', left_on=['investors_name','investor_id'], right_on = ['node','index'])


In [106]:
nodes

,node,index,country,continent,industry,bipartite,id
0,Shionogi & Co.,262824.0,Japan,Asia,Pharmaceuticals / Drugs,1,0
1,Astellas Pharma,243168.0,Japan,Asia,Pharmaceuticals / Drugs,1,1
2,Sumitomo Dainippon Pharma,447180.0,Japan,Asia,Pharmaceuticals / Drugs,1,2
3,Konica Minolta,57191.0,Japan,Asia,Medical Devices & Equipment,1,3
4,Takeda Pharmaceutical,373007.0,Japan,Asia,Pharmaceuticals / Drugs,1,4
...,...,...,...,...,...,...,...
83253,Shenzhen Qianhai Beizeng Asset Management Co. ...,179782.0,China,Asia,Venture Capital,0,83253
83254,Wuxi High-Tech District Seed Fund,189318.0,China,Asia,Other,0,83254
83255,Edvantage Group,190377.0,China,Asia,Corporation,0,83255
83256,Healgen,NaN,NaN,NaN,NaN,0,83256


In [81]:
nodes_fda = pd.read_csv('xi.csv')['x']
covariates = pd.read_csv('CB_with_covariates.csv')

C:\Users\Marco\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (7,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [98]:
#aggiungere min max mediana

eigen_max = []
eigen_min = []

pages_max = []
pages_min = []

degcen_max = []
degcen_min = []

bet_max = []
bet_min = []

avg_max = []
avg_min = []

clo_max = []
clo_min = []

newman_max = []
newman_min = []

vote_min = []
vote_max = []

sub_max = []
sub_min = []

dis_mean = []

number = []
sub0 = cb_new[['target_ID','date_y','id_y', 'size_real']]

for i in tqdm(nodes_fda):
    
    sub1 = sub0[sub0['target_ID']==i]
    sub1 = sub1.dropna()
    sub2 = sub1[sub1['date_y']==sub1['date_y'].values[0]]
    anno = sub2['date_y'].values[0].year
    investors = set(sub2['id_y'])
    eigens = []
    pages = []
    degcen = []
    bet = []
    avg = []
    dis = []
    clo = []
    newman = []
    vote = []
    sub = []
    num = []
    for l in investors:
        try:
            eigens.append(covariates[covariates['id']==l]['eigenvector_centrality_single_'+str(anno)].values[0])
        except:
            eigens.append(np.nan)
        try:
            pages.append(covariates[covariates['id']==l]['pagerank_single_'+str(anno)].values[0])
        except:
            pages.append(np.nan)
        try:
            degcen.append(covariates[covariates['id']==l]['degree_centrality_single_'+str(anno)].values[0]) 
        except:
            degcen.append(np.nan)
        try:
            bet.append(covariates[covariates['id']==l]['betweenness_centrality_single_'+str(anno)].values[0]) 
        except:
            bet.append(np.nan)
        try:
            avg.append(covariates[covariates['id']==l]['average_neighbor_degree_single_'+str(anno)].values[0])
        except:
            avg.append(np.nan)
        try:
            dis.append(covariates[covariates['id']==l]['dispersion_single_'+str(anno)].values[0])
        except:
            dis.append(np.nan) 
        try:
            clo.append(covariates[covariates['id']==l]['closeness_centrality_single_'+str(anno)].values[0])
        except:
            clo.append(np.nan)
        try:
            newman.append(covariates[covariates['id']==l]['newman_betweenness_centrality_single_'+str(anno)].values[0])
        except:
            newman.append(np.nan)
        try:
            vote.append(covariates[covariates['id']==l]['voterank_single_'+str(anno)].values[0])
        except:
            vote.append(np.nan)
        try:
            sub.append(covariates[covariates['id']==l]['subgraph_centrality_single_' +str(anno)].values[0])
        except:
            sub.append(np.nan)
        try:
            num.append(len(investors))
        except: 
            num.append(len(investors))
          
       
        
        
       
            
    eigen_max.append(np.nanmax(eigens))
    eigen_min.append(np.nanmin(eigens))
    
    pages_max.append(np.nanmax(pages))
    pages_min.append(np.nanmin(pages))
    
    degcen_max.append(np.nanmax(degcen))
    degcen_min.append(np.nanmin(degcen))
    
    bet_max.append(np.nanmax(bet))
    bet_min.append(np.nanmin(bet))
    
    avg_max.append(np.nanmax(avg))
    avg_min.append(np.nanmax(avg))
    
    #dis_mean.append(np.nanmean(dis))
    
    clo_max.append(np.nanmax(clo))
    clo_min.append(np.nanmin(clo))
    
    newman_max.append(np.nanmax(newman))
    newman_min.append(np.nanmin(newman))
    
    vote_min.append(np.nanmin(vote))
    vote_max.append(np.nanmax(vote))
    
    sub_max.append(np.nanmax(sub))
    sub_min.append(np.nanmin(sub))
    
    number.append(num)

<ipython-input-98-edd6bb860d66>:104: RuntimeWarning: All-NaN axis encountered
  eigen_max.append(np.nanmax(eigens))
<ipython-input-98-edd6bb860d66>:105: RuntimeWarning: All-NaN axis encountered
  eigen_min.append(np.nanmin(eigens))
<ipython-input-98-edd6bb860d66>:107: RuntimeWarning: All-NaN axis encountered
  pages_max.append(np.nanmax(pages))
<ipython-input-98-edd6bb860d66>:108: RuntimeWarning: All-NaN axis encountered
  pages_min.append(np.nanmin(pages))
<ipython-input-98-edd6bb860d66>:110: RuntimeWarning: All-NaN axis encountered
  degcen_max.append(np.nanmax(degcen))
<ipython-input-98-edd6bb860d66>:111: RuntimeWarning: All-NaN axis encountered
  degcen_min.append(np.nanmin(degcen))
<ipython-input-98-edd6bb860d66>:113: RuntimeWarning: All-NaN axis encountered
  bet_max.append(np.nanmax(bet))
<ipython-input-98-edd6bb860d66>:114: RuntimeWarning: All-NaN axis encountered
  bet_min.append(np.nanmin(bet))
<ipython-input-98-edd6bb860d66>:116: RuntimeWarning: All-NaN axis encountered
  av

In [108]:
covariate = nodes[(nodes['index'].isin(list(nodes_fda)))&(nodes['bipartite']==1)]

In [113]:
x = number
y = [a[0] for a in number]

1

In [121]:
covariate = nodes[(nodes['index'].isin(list(nodes_fda)))&(nodes['bipartite']==1)]
covariate = covariate.assign(eigen_max = eigen_max, eigen_min = eigen_min, 
                             pages_max = pages_max, pages_min = pages_min,
                             degcen_max = degcen_max, degcen_min = degcen_min,
                             bet_max = bet_max, bet_min = bet_min, 
                             avg_max = avg_max, avg_min = avg_min,
                             #dis_mean = dis_mean, 
                             newman_max = newman_max, newman_min = newman_min, 
                             vote_min = vote_min, vote_max = vote_max,
                             sub_max = sub_max, sub_min = sub_min,
                             clo_max = clo_max, clo_min = clo_min,
                             num = y)


In [124]:
covariate

,node,index,country,continent,industry,bipartite,id,eigen_max,eigen_min,pages_max,...,avg_min,newman_max,newman_min,vote_min,vote_max,sub_max,sub_min,clo_max,clo_min,num
15,NeuroMolecular Pharmaceuticals,20125.0,United States,North America,Pharmaceuticals / Drugs,1,15,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
17,AngioScore,18536.0,United States,North America,Medical Devices & Equipment,1,17,0.101616,0.001983,0.003299,...,217.000000,0.027182,0.000000,10.0,46.0,1.230000e+39,4.700000e+35,0.425625,0.304958,4
19,Prematics,18285.0,United States,North America,Healthcare Software,1,19,0.104045,0.006769,0.003282,...,79.069444,0.023125,0.000007,10.0,1456.0,9.870000e+25,4.180000e+23,0.408753,0.309002,9
26,Emphasys Medical,21524.0,United States,North America,Medical Devices & Equipment,1,26,0.000263,0.000263,0.000235,...,22.571429,0.000208,0.000208,913.0,913.0,8.260000e+33,8.260000e+33,0.265837,0.265837,1
27,ev3,21526.0,United States,North America,Medical Devices & Equipment,1,27,0.019302,0.000247,0.001343,...,71.831579,0.008337,0.000000,71.0,539.0,4.450000e+37,7.260000e+33,0.368942,0.266811,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47344,Zuoli Baicao,595419.0,China,Asia,Pharmaceuticals / Drugs,1,47344,0.000039,0.000039,0.000114,...,9.000000,0.000000,0.000000,2696.0,2696.0,6.653400e+15,6.653400e+15,0.181571,0.181571,1
47352,Lifeon Pharmaceutical,595679.0,China,Asia,Pharmaceuticals / Drugs,1,47352,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
47387,NovAliX,411293.0,France,Europe,Drug Discovery,1,47387,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
47421,Clinicom,656337.0,United States,North America,Mobile Software & Services,1,47421,0.152020,0.001868,0.002338,...,219.000000,0.039321,0.000000,0.0,2357.0,9.190000e+34,1.390000e+31,0.331625,0.240107,3


In [48]:
sub0 = cb_new[['target_ID','date_y','id_y', 'size_real']]

In [125]:
covariate.to_csv('covariate2.csv', index = False)

In [78]:
sub0 = cb_new[['target_ID','date_y','id_y', 'size_real' ]]
sub1 = sub0[sub0['target_ID']==64]
sub1 = sub1.dropna()
sub2 = sub1[sub1['date_y']==sub1['date_y'].values[0]]
anno = sub2['date_y'].values[0].year
investors = set(sub2['id_y'])

In [80]:
investors

{50468, 50475, 50519, 50594, 50667, 51418, 54785, 55854, 56371}

In [5]:
def apply_function_network(lista_storage,lista_nomi,funzione,directed=False):
    for year in tqdm(range(1990,2019)):
        string='orgorg4_single'+str(year)+'.gpickle'
        network=nx.readwrite.read_gpickle(string)
        storage=str(funzione)+'_orgorg3_single'+str(year)
        lista_storage.append(funzione(network))
        lista_nomi.append(storage)
#         network=nx.readwrite.read_gpickle('bidbid_stage_self_cb'+str(year)+'.gpickle')
#         lista_storage.append(funzione(network))
#         lista_nomi.append(str(funzione)+'_bidder_'+str(year))
    return lista_storage,lista_nomi
def apply_function_network_community(lista_storage,lista_nomi,funzione,directed=False):
    for year in tqdm(range(1999,2020)):
        string='orgorg3_single_/orgorg3_single'+str(year)+'.gpickle'
        network=nx.readwrite.read_gpickle(string)
        storage='Louvain_orgorg3_single'+str(year)
        lista_storage.append(funzione(network))
        lista_nomi.append(storage)
    #network=nx.readwrite.read_gpickle('bidbid_stage_single_cb.gpickle')
    #lista_storage.append(funzione(network))
    #lista_nomi.append(str(funzione)+'_single')
    return lista_storage,lista_nomi

In [3]:
lista_funzioni_def=[nx.betweenness_centrality,nx.harmonic_centrality,nx.clustering, nx.core_number,nx.eigenvector_centrality, nx.degree,nx.closeness_centrality,nx.pagerank,nx.load_centrality,nx.average_neighbor_degree]



In [6]:
lista_storage,lista_nomi=apply_function_network(lista_storage,lista_nomi,nx.betweenness_centrality)

  0%|          | 0/29 [00:00<?, ?it/s]

In [4]:
# Array di funzioni da applicare - ciclo for su di esse
#lista_funzioni=[nx.voterank, nx.eigenvector_centrality, nx.degree_centrality, nx.pagerank, nx.closeness_centrality, nx.incremental_closeness_centrality, nx.load_centrality, nx.subgraph_centrality, nx.dispersion, nx.percolation_centrality,nx.average_neighbor_degree,nx.constraint, nx.betweenness_centrality]
# lista_funzioni2=[nx.dispersion,nx.constraint,nx.betweenness_centrality, nx.trophic_levels, nx.harmonic_centrality, nx.enumerate_all_cliques, nx.clustering, nx.core_number, nx.rich_club_coefficient, nx.closeness_vitality]
# lista_funzioni_rifare=[nx.harmonic_centrality, nx.enumerate_all_cliques, nx.clustering,nx.core_number,nx.closeness_vitality]
# Funzione prende df, se direzionato o meno
# lista_funzioni_prova2=[nx.eigenvector_centrality, nx.degree]
lista_storage=[]
lista_nomi=[]
for funzione in lista_funzioni_def:
    try:
        print('Trying'+str(funzione))
        lista_storage2, lista_nomi2=apply_function_network(lista_storage2,lista_nomi2,nx.betweenness_centrality)
    except:
        print(str(funzione)+' did not work')
        continue

Trying<function betweenness_centrality at 0x7fe0d0398dc0>
<function betweenness_centrality at 0x7fe0d0398dc0> did not work
Trying<function harmonic_centrality at 0x7fe0d03bc4c0>
<function harmonic_centrality at 0x7fe0d03bc4c0> did not work
Trying<function clustering at 0x7fe0989614c0>
<function clustering at 0x7fe0989614c0> did not work
Trying<function core_number at 0x7fe098973af0>
<function core_number at 0x7fe098973af0> did not work
Trying<function eigenvector_centrality at 0x7fe0d03bc3a0>
<function eigenvector_centrality at 0x7fe0d03bc3a0> did not work
Trying<function degree at 0x7fe10090ac10>
<function degree at 0x7fe10090ac10> did not work
Trying<function closeness_centrality at 0x7fe0d03a3a60>
<function closeness_centrality at 0x7fe0d03a3a60> did not work
Trying<function pagerank at 0x7fe1009fc310>
<function pagerank at 0x7fe1009fc310> did not work
Trying<function newman_betweenness_centrality at 0x7fe0d03c71f0>
<function newman_betweenness_centrality at 0x7fe0d03c71f0> did not 

In [4]:
network=nx.readwrite.read_gpickle('orgorg4_single2006.gpickle')

In [5]:
def apply_function_network(funzione):
    for year in tqdm(range(1990,2021)):
        string2='orgorg4_single'+str(year)+'.gpickle'
        network2=nx.readwrite.read_gpickle(string2)
        storage2=str(funzione)+'_orgorg4_single'+str(year)+'.csv'
        covariate2 = funzione(network2)
        covariate3 = pd.Series(covariate2)
        print('Arrivo qui', year)
        covariate3.to_csv(storage2)
        print('Done', funzione, 'for year', year)

In [ ]:
for function in lista_funzioni_def:
    apply_function_network(function)

  0%|          | 0/31 [00:00<?, ?it/s]

Arrivo qui 1990
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1990
Arrivo qui 1991
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1991
Arrivo qui 1992
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1992
Arrivo qui 1993
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1993
Arrivo qui 1994
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1994
Arrivo qui 1995
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1995
Arrivo qui 1996
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1996
Arrivo qui 1997
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1997
Arrivo qui 1998
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1998
Arrivo qui 1999
Done <function betweenness_centrality at 0x7fa37894dee0> for year 1999
Arrivo qui 2000
Done <function betweenness_centrality at 0x7fa37894dee0> for year 2000
Arrivo qui 2001
Done <function betweenness_

In [9]:
import concurrent
with concurrent.futures.ProcessPoolExecutor() as executor:
    # results = [executor.submit(apply_function_network,funzione) for funzione in lista_funzioni]
    executor.map(apply_function_network, lista_funzioni_def)

BrokenProcessPool: A child process terminated abruptly, the process pool is not usable anymore

In [7]:
lista_storage2=[]
lista_nomi2=[]
apply_function_network(lista_storage2,lista_nomi2,nx.degree)

NameError: name 'apply_function_network' is not defined

In [11]:
lista_nomi2[10]

'<function degree at 0x7fb1412bf9d0>_orgorg3_single2000'